In [70]:
import gurobipy as gp
import numpy as np

In [69]:
coefficient = dict()
coefficient['y'] = -np.r_[[1.045], [0] * 10]
coefficient['x'] = -np.r_[1.01:1.10:0.01]
coefficient['b'] = np.r_[[1000], np.r_[[100] * 10]]

In [75]:
coefficient['x']

array([-1.01, -1.02, -1.03, -1.04, -1.05, -1.06, -1.07, -1.08, -1.09,
       -1.1 ])

In [72]:
from typing import Any

# 继承type，使Singleton成为元类，以Singleton为metaclass的类在实例化的时候调用Singleton的__call__()函数
class Singleton(type):
    _instance = {}

    def __call__(cls, *args: Any, **kwds: Any) -> Any:
        if cls not in cls._instance:
            cls._instance[cls] = super(Singleton, cls).__call__(*args,  **kwds)
        return cls._instance[cls]

In [85]:
from typing import Dict

class MP(metaclass=Singleton):
    def __init__(self, coefficient: Dict[str, np.ndarray]) -> None:
        self.master_problem = gp.Model("Master Problem")
        self.master_problem.setParam('OutputFlag', 0)
        self.y = np.array(self.master_problem.addMVar(1, vtype=gp.GRB.INTEGER, name="y").tolist())
        self.y = np.concatenate([self.y, self.master_problem.addMVar(coefficient['x'].shape, vtype=gp.GRB.INTEGER, lb=0, ub=0, name='y_pad').tolist()]).tolist()
        self.y = gp.MVar.fromlist(self.y)
        
        self.q = self.master_problem.addMVar(1, lb=-float("inf"), vtype=gp.GRB.CONTINUOUS, name="q")
        self.master_problem.setObjective((coefficient['y'] * self.y).sum() + 1* self.q, sense=gp.GRB.MINIMIZE)
        self.b = coefficient['b']
        self.rhs = coefficient["b"] - self.y
    def update_constr(self, extrem_pnt: np.ndarray = None, extrem_ray: np.ndarray = None):
        if extrem_pnt or extrem_ray:
            # 可行约束
            self.master_problem.addConstr(extrem_ray.dot(self.rhs) <= 0, name='feasible_constr')
            # 最优性约束
            self.master_problem.addConstr(extrem_pnt.dot(self.rhs) - self.q <= 0, name='optimal_constr')
    def solve(self):
        self.master_problem.optimize()
        return self.y.X[0]

In [159]:
from typing import Dict

from networkx import constraint

class SUBP(metaclass=Singleton):
    def __init__(self, coefficient: Dict[str, np.ndarray], ) -> None:
        self.sub_problem = gp.Model("Sub Problem")
        self.sub_problem.setParam('OutputFlag', 0)
        self.x = self.sub_problem.addMVar(coefficient['x'].shape, vtype=gp.GRB.CONTINUOUS, lb=0, ub=100, name="x")
        self.sub_problem.setObjective((coefficient['x'] * self.x).sum())
    def update_y_value(self, y_value: int):
        self.y_value = y_value
        self.sub_problem.addConstr(self.x.sum() - 1000 + self.y_value <= 0, name='sub_problem_constr')
    def solve(self):
        self.sub_problem.setParam('InfUnbdInfo', 1)
        self.sub_problem.optimize()
        if self.sub_problem.Status != 2:
            constraints = self.sub_problem.getConstrs()
            for c in constraints:
                print(c.farkasdual)

In [133]:
mp = MP(coefficient)
y_value = mp.solve()
y_value

1e+30

In [160]:
sub_problem = SUBP(coefficient)
sub_problem.update_y_value(1500)
print(sub_problem.solve())

1.0
None


In [19]:
y_value = 0
sub_problem = gp.Model("Sub Problem")
x = sub_problem.addMVar(coefficient['x'].shape, vtype=gp.GRB.CONTINUOUS, name="x")
sub_problem.addConstr(x.sum() - 1000 + y_value <= 0, name='sub_problem_constr')
sub_problem.setObjective(coefficient['x'] * x, gp.GRB.MAXIMIZE)
# coefficient['x'] * x

<MLinExpr (10,)  >
array([ -1.01 <gurobi.Var *Awaiting Model Update*>,
        -1.02 <gurobi.Var *Awaiting Model Update*>,
        -1.03 <gurobi.Var *Awaiting Model Update*>,
        -1.04 <gurobi.Var *Awaiting Model Update*>,
        -1.05 <gurobi.Var *Awaiting Model Update*>,
        -1.06 <gurobi.Var *Awaiting Model Update*>,
        -1.07 <gurobi.Var *Awaiting Model Update*>,
        -1.08 <gurobi.Var *Awaiting Model Update*>,
        -1.09 <gurobi.Var *Awaiting Model Update*>,
        -1.1 <gurobi.Var *Awaiting Model Update*>])